In [5]:
import psycopg2
from config import load_config
import pandas as pd
import folium
import os
from sklearn.cluster import DBSCAN
from geopy.distance import geodesic
import numpy as np
import json

In [6]:
def capturar_data_mapa(conn, table_name, linha):
    cur = conn.cursor()
    all_data = [] 
    try:
        fetch_query = f'''
        SELECT latitude::double precision, longitude::double precision
        FROM {table_name}
        WHERE linha='{linha}'
        '''
        cur.execute(fetch_query)
        rows = cur.fetchall()
        all_data.extend(rows)
    except Exception as e:
            print(f"Erro ao executar a query na tabela {table_name}: {e}")
            conn.rollback()
    else:
         conn.commit()
    cur.close()
    return all_data

In [7]:
def limpar_dados_tabela(table_name, conn):
    cur = conn.cursor()
    try:
        # Adicionar a coluna hour, se não existir
        cur.execute(f"ALTER TABLE {table_name} ADD COLUMN IF NOT EXISTS hour INTEGER;")
        
        # Atualizar a coluna hour com base no datahora
        cur.execute(f"UPDATE {table_name} SET hour = EXTRACT(HOUR FROM TO_TIMESTAMP(datahora / 1000));")
        
        # Criar uma nova tabela filtrada
        cur.execute(f"""
            CREATE TABLE {table_name}_filter AS
            SELECT *
            FROM {table_name}
            WHERE hour BETWEEN 8 AND 22;
        """)
        
        # Remover a coluna hour das tabelas
        cur.execute(f"ALTER TABLE {table_name} DROP COLUMN hour;")
        cur.execute(f"ALTER TABLE {table_name}_filter DROP COLUMN hour;")
        
        # Commit das alterações
        conn.commit()
        print(f"Alterações na tabela {table_name} foram comitadas com sucesso.")
    except Exception as e:
        conn.rollback()
        print(f"Erro ao limpar dados da tabela {table_name}: {e}")
    finally:
        cur.close()
        print(f"Cursor fechado para a tabela {table_name}.")

In [8]:
def adicionar_geom(table_name, conn):
    cur = conn.cursor()
    try:
        # Adicionar a coluna geom se não existir
        cur.execute(f"""
            ALTER TABLE {table_name} ADD COLUMN IF NOT EXISTS geom geography(Point, 4326);
        """)
        
        # Atualizar a coluna geom com os valores de longitude e latitude
        cur.execute(f"""
            UPDATE {table_name}
            SET geom = ST_SetSRID(ST_MakePoint(longitude::double precision, latitude::double precision), 4326);
        """)
        
        # Commit das alterações
        conn.commit()
        print(f"Coluna geom criada e preenchida na tabela {table_name}_filter com sucesso.")
    except Exception as e:
        # Rollback em caso de erro
        conn.rollback()
        print(f"Erro ao atualizar a tabela {table_name}_filter: {e}")
    finally:
        # Fechar o cursor
        cur.close()
        print(f"Cursor fechado para a tabela {table_name}_filter.")

In [9]:
def filtra_linhas(table_name, linhas_de_interesse, conn):
    cur = conn.cursor()
    temp_table_name = table_name + '_intermediate'
    linhas_str = ', '.join([f"'{linha}'" for linha in linhas_de_interesse])
    try:
        # Criar tabela intermediária filtrada
        cur.execute(f"""
            CREATE TABLE {temp_table_name} AS
            SELECT *
            FROM {table_name}_filter
            WHERE linha IN ({linhas_str});
        """)
        print(f"Tabela intermediária {temp_table_name} criada com sucesso.")
        conn.commit()
        print(f"Alterações na tabela {temp_table_name} foram comitadas com sucesso.")
    except Exception as e:
        print(f"Erro ao criar a tabela intermediária {temp_table_name}: {e}")
        conn.rollback()
    finally:
        cur.close()
        print(f"Cursor fechado para a tabela {temp_table_name}.")

In [10]:
def sobrescreve_tabelas(table_name, conn):
    cur = conn.cursor()
    temp_table_name = table_name + '_intermediate'
    try:
        # Verificar se a tabela temporária existe
        cur.execute(f"""
            SELECT EXISTS (
                SELECT FROM pg_tables
                WHERE schemaname = 'public' AND tablename = '{temp_table_name}'
            );
        """)
        exists = cur.fetchone()[0]

        if exists:
            # Excluir a tabela original
            cur.execute(f"DROP TABLE IF EXISTS {table_name}_filter;")
            # Renomear a tabela temporária para o nome original
            cur.execute(f"ALTER TABLE {temp_table_name} RENAME TO {table_name}_filter;")
            print(f"Tabela {table_name}_filter sobrescrita com sucesso.")
            conn.commit()
            print(f"Tabela {table_name}_filter comitada com sucesso.")
        else:
            print(f"Tabela temporária {temp_table_name} não encontrada.")
            conn.rollback()
    except Exception as e:
        print(f"Erro ao sobrescrever a tabela {table_name}_filter: {e}")
        conn.rollback()
    finally:
        cur.close()
        print(f"Cursor fechado para a tabela {table_name}_filter.")


In [11]:
def margem_erro(coord1, coord2, radius=100):
    return geodesic(coord1, coord2).meters <= radius

In [12]:
def criar_tabela_destino(conn):
    cur = conn.cursor()
    try:
        cur.execute("""
            CREATE TABLE IF NOT EXISTS combined_table_all AS
            SELECT * FROM dia_0105 WHERE 1=0;
        """)
        conn.commit()
        print("Tabela de destino 'combined_table' criada com sucesso.")
    except Exception as e:
        conn.rollback()
        print(f"Erro ao criar a tabela de destino: {e}")
    finally:
        cur.close()

In [13]:
def inserir_dados(table_name, conn):
    cur = conn.cursor()
    try:
        cur.execute(f"INSERT INTO combined_table_all SELECT * FROM {table_name};")
        conn.commit()
        print(f"Dados inseridos na tabela 'combined_table' a partir de '{table_name}' com sucesso.")
    except Exception as e:
        conn.rollback()
        print(f"Erro ao inserir dados da tabela {table_name}: {e}")
    finally:
        cur.close()

In [14]:
def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)


In [15]:
config = load_config()
conn = connect(config)

Connected to the PostgreSQL server.


In [12]:
linhas_de_interesse = ['483', '864', '639', '3', '309', '774', '629', '371', '397', '100', '838', '315', '624', '388', '918', '665', '328', '497', '878', '355', '138', '606', '457', '550', '803', '917', '638', '2336', '399', '298', '867', '553', '565', '422', '756', '186012003', '292', '554', '634', '232', '415', '2803', '324', '852', '557', '759', '343', '779', '905', '108']

In [13]:
table_names = ['dia_0105', 'dia_0205', 'dia_0305', 'dia_0405', 'dia_0505', 'dia_0605', 'dia_0705', 'dia_0805', 'dia_0905', 'dia_1005', 'dia_2504', 'dia_2604', 'dia_2704', 'dia_2804', 'dia_2904', 'dia_3004']

In [157]:
"""for table in table_names:
    limpar_dados_tabela(table, conn)"""

'for table in table_names:\n    limpar_dados_tabela(table, conn)'

In [158]:
"""for table in table_names:
    adicionar_geom(table, conn)"""

Coluna geom criada e preenchida na tabela dia_0105_filter com sucesso.
Cursor fechado para a tabela dia_0105_filter.
Coluna geom criada e preenchida na tabela dia_0205_filter com sucesso.
Cursor fechado para a tabela dia_0205_filter.
Coluna geom criada e preenchida na tabela dia_0305_filter com sucesso.
Cursor fechado para a tabela dia_0305_filter.
Coluna geom criada e preenchida na tabela dia_0405_filter com sucesso.
Cursor fechado para a tabela dia_0405_filter.
Coluna geom criada e preenchida na tabela dia_0505_filter com sucesso.
Cursor fechado para a tabela dia_0505_filter.
Coluna geom criada e preenchida na tabela dia_0605_filter com sucesso.
Cursor fechado para a tabela dia_0605_filter.
Coluna geom criada e preenchida na tabela dia_0705_filter com sucesso.
Cursor fechado para a tabela dia_0705_filter.
Coluna geom criada e preenchida na tabela dia_0805_filter com sucesso.
Cursor fechado para a tabela dia_0805_filter.
Coluna geom criada e preenchida na tabela dia_0905_filter com su

In [142]:
"""for table in table_names:
    filtra_linhas(table, linhas_de_interesse, conn)
    sobrescreve_tabelas(table, conn)"""

Tabela intermediária dia_0105_intermediate criada com sucesso.
Alterações na tabela dia_0105_intermediate foram comitadas com sucesso.
Cursor fechado para a tabela dia_0105_intermediate.
Tabela dia_0105_filter sobrescrita com sucesso.
Tabela dia_0105_filter comitada com sucesso.
Cursor fechado para a tabela dia_0105_filter.
Tabela intermediária dia_0205_intermediate criada com sucesso.
Alterações na tabela dia_0205_intermediate foram comitadas com sucesso.
Cursor fechado para a tabela dia_0205_intermediate.
Tabela dia_0205_filter sobrescrita com sucesso.
Tabela dia_0205_filter comitada com sucesso.
Cursor fechado para a tabela dia_0205_filter.
Tabela intermediária dia_0305_intermediate criada com sucesso.
Alterações na tabela dia_0305_intermediate foram comitadas com sucesso.
Cursor fechado para a tabela dia_0305_intermediate.
Tabela dia_0305_filter sobrescrita com sucesso.
Tabela dia_0305_filter comitada com sucesso.
Cursor fechado para a tabela dia_0305_filter.
Tabela intermediária d

In [136]:
"""for linha in linhas_de_interesse:
    data = capturar_data_mapa(conn, 'dia_2704_filter', linha)
    if data:
        avg_latitude = sum([d[0] for d in data]) / len(data)
        avg_longitude = sum([d[1] for d in data]) / len(data)
        m = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=12)
        # Adicione os pontos ao mapa
        for lat, lon in data:
            folium.CircleMarker(location=[lat, lon], radius=1, color='blue').add_to(m)
        output_path = os.path.join(f'trajetos/output_map_{linha}.html')
        m.save(output_path)
        print(f"Mapa salvo em {output_path}")"""

'for linha in linhas_de_interesse:\n    data = capturar_data_mapa(conn, \'dia_2704_filter\', linha)\n    if data:\n        avg_latitude = sum([d[0] for d in data]) / len(data)\n        avg_longitude = sum([d[1] for d in data]) / len(data)\n        m = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=12)\n        # Adicione os pontos ao mapa\n        for lat, lon in data:\n            folium.CircleMarker(location=[lat, lon], radius=1, color=\'blue\').add_to(m)\n        output_path = os.path.join(f\'trajetos/output_map_{linha}.html\')\n        m.save(output_path)\n        print(f"Mapa salvo em {output_path}")'

In [14]:
"""criar_tabela_destino(conn)
for table in table_names:
    inserir_dados(table, conn)"""

Tabela de destino 'combined_table' criada com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0105_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0205_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0305_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0405_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0505_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0605_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0705_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0805_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0905_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_1005_filter' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_2504_filter' com sucesso.

In [23]:
pontos_de_parada = {}

In [83]:
query = """
WITH ordered_points AS (
    SELECT
        ordem,
        linha,
        geom,
        TO_TIMESTAMP(datahora / 1000) AS datahora_ts,
        LAG(TO_TIMESTAMP(datahora / 1000)) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_datahora_ts,
        LAG(geom) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_geom,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS rn
    FROM combined_table
),
same_position_periods AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        EXTRACT(EPOCH FROM (datahora_ts - prev_datahora_ts)) AS duration,
        CASE 
            WHEN ST_DWithin(geom, prev_geom, 15) THEN 1
            ELSE 0
        END AS is_stationary,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY datahora_ts) - 
        ROW_NUMBER() OVER (PARTITION BY ordem, CASE WHEN ST_DWithin(geom, prev_geom, 15) THEN 1 ELSE 0 END ORDER BY datahora_ts) AS grp
    FROM ordered_points
),
stationary_groups AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        duration,
        is_stationary,
        grp
    FROM same_position_periods
    WHERE is_stationary = 1
)
SELECT
    ordem,
    linha,
    MIN(prev_datahora_ts) AS start_time,
    MAX(datahora_ts) AS end_time,
    SUM(duration) AS total_duration,
    ST_X(geom::geometry) AS longitude,
    ST_Y(geom::geometry) AS latitude,
    grp
FROM stationary_groups
GROUP BY ordem, linha, grp, geom
HAVING SUM(duration) >= 600; -- 10 minutes in seconds
"""

In [84]:
df = pd.read_sql(query, conn)

/tmp/ipykernel_7675/1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [85]:
df_unique = df.drop_duplicates(subset=['grp'])

In [86]:
for linha in linhas_de_interesse:
    pontos_de_parada[linha]=[]
    for _, row in df_unique.loc[df_unique['linha'] == linha].iterrows():
        lat, lon = row['latitude'], row['longitude']
        ponto = (lat, lon)
        encontrado = False
        if row['total_duration'] > 2400:
            continue
        for pontos in pontos_de_parada[linha]:
            if margem_erro(pontos[0], ponto):
                encontrado = True
                pontos[1] += 1
                break
        if not encontrado:
            pontos_de_parada[linha].append([ponto, 1])
for linha in pontos_de_parada.keys():
        pontos_de_parada[linha] = sorted(pontos_de_parada[linha], key=lambda x: x[1], reverse=True)

In [87]:
pontos_finais = {}

In [88]:
for linha in linhas_de_interesse:
    print(f"{linha} {pontos_de_parada[linha][:4]}")

483 [[(-22.80462, -43.3097), 73], [(-22.83695, -43.28397), 44], [(-22.90331, -43.17292), 5], [(-22.98572, -43.19701), 5]]
864 [[(-22.89371, -43.53251), 49], [(-22.89434, -43.53179), 16], [(-22.90285, -43.55566), 7], [(-22.89527, -43.53218), 4]]
639 [[(-22.81698, -43.30219), 204], [(-22.81109, -43.32993), 63], [(-22.88643, -43.29749), 3], [(-22.91696, -43.25807), 2]]
3 [[(-22.88441, -43.49511), 72], [(-22.88326, -43.49577), 16], [(-22.86105, -43.45404), 2], [(-22.90092, -43.55167), 1]]
309 [[(-22.87422, -43.24181), 147], [(-22.90459, -43.19256), 142], [(-22.87557, -43.24144), 114], [(-23.00197, -43.36538), 20]]
774 [[(-22.81676, -43.3018), 65], [(-22.80744, -43.32766), 42], [(-22.81488, -43.30976), 1], [(-22.828, -43.32254), 1]]
629 [[(-22.81685, -43.30224), 92], [(-22.82503, -43.33258), 19], [(-22.92302, -43.23276), 4], [(-22.75362, -43.29507), 4]]
371 [[(-22.90137, -43.34716), 45], [(-22.90853, -43.18917), 6], [(-22.88599, -43.25172), 1], [(-22.87704, -43.36831), 1]]
397 [[(-22.90219,

397
388
759
138 
tiveram resultados inconclusivos, adicionarei manualmente os pontos finais.

In [89]:
for linha in linhas_de_interesse:
    pontos_de_parada_linha = pontos_de_parada[linha]
    if len(pontos_de_parada[linha]) > 0:
        if (pontos_de_parada_linha[1][1] - pontos_de_parada_linha[2][1]) < 0.2*pontos_de_parada_linha[2][1]:
            d2 = geodesic(pontos_de_parada_linha[0][0], pontos_de_parada_linha[1][0]).meters
            d3 = geodesic(pontos_de_parada_linha[0][0], pontos_de_parada_linha[2][0]).meters
            if d2 > d3:
                pontos_finais[linha] = [pontos_de_parada_linha[0][0], pontos_de_parada_linha[1][0]]
            else:
                pontos_finais[linha] = [pontos_de_parada_linha[0][0], pontos_de_parada_linha[2][0]]
        else:
            pontos_finais[linha] = [pontos_de_parada_linha[0][0], pontos_de_parada_linha[1][0]]

In [90]:
for linha in pontos_finais.keys():
    print(f"{linha} {pontos_finais[linha]}")

483 [(-22.80462, -43.3097), (-22.83695, -43.28397)]
864 [(-22.89371, -43.53251), (-22.89434, -43.53179)]
639 [(-22.81698, -43.30219), (-22.81109, -43.32993)]
3 [(-22.88441, -43.49511), (-22.88326, -43.49577)]
309 [(-22.87422, -43.24181), (-22.90459, -43.19256)]
774 [(-22.81676, -43.3018), (-22.80744, -43.32766)]
629 [(-22.81685, -43.30224), (-22.82503, -43.33258)]
371 [(-22.90137, -43.34716), (-22.90853, -43.18917)]
397 [(-22.90219, -43.55503), (-22.84356, -43.25076)]
100 [(-22.90455, -43.19202), (-22.88857, -43.2932)]
838 [(-22.89452, -43.53222), (-22.9029, -43.55572)]
315 [(-22.87521, -43.24173), (-23.02203, -43.48246)]
624 [(-22.82872, -43.41937), (-22.91151, -43.21217)]
388 [(-22.93533, -43.65589), (-22.90076, -43.17704)]
918 [(-22.8945, -43.49018), (-22.89338, -43.53211)]
665 [(-22.80609, -43.36326), (-22.922, -43.23262)]
328 [(-22.78881, -43.16222), (-22.81469, -43.18696)]
497 [(-22.83643, -43.28357), (-22.80453, -43.30981)]
878 [(-22.88036, -43.35735), (-22.91579, -43.36082)]
35

In [91]:
pontos_finais['397'] = [(-22.90202, -43.55532), (-22.90121, -43.17778)]
pontos_finais['388'] = [(-22.93554, -43.65626), (-22.90121, -43.17778)]
pontos_finais['759'] = [(-22.93554, -43.65626), (-22.83152, -43.34393)]

In [138]:
criar_tabela_destino(conn)
for table in table_names:
    inserir_dados(table, conn)

Tabela de destino 'combined_table' criada com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0105' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0205' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0305' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0405' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0505' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0605' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0705' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0805' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_0905' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_1005' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_2504' com sucesso.
Dados inseridos na tabela 'combined_table' a partir de 'dia_2604' com sucess

In [140]:
query_garagem = """
WITH ordered_points AS (
    SELECT
        ordem,
        linha,
        geom,
        TO_TIMESTAMP(datahora / 1000) AS datahora_ts,
        LAG(TO_TIMESTAMP(datahora / 1000)) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_datahora_ts,
        LAG(geom) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_geom,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS rn
    FROM combined_table_all
),
same_position_periods AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        EXTRACT(EPOCH FROM (datahora_ts - prev_datahora_ts)) AS duration,
        CASE 
            WHEN ST_DWithin(geom, prev_geom, 15) THEN 1
            ELSE 0
        END AS is_stationary,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY datahora_ts) - 
        ROW_NUMBER() OVER (PARTITION BY ordem, CASE WHEN ST_DWithin(geom, prev_geom, 15) THEN 1 ELSE 0 END ORDER BY datahora_ts) AS grp
    FROM ordered_points
),
stationary_groups AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        duration,
        is_stationary,
        grp
    FROM same_position_periods
    WHERE is_stationary = 1
)
SELECT
    ordem,
    linha,
    MIN(prev_datahora_ts) AS start_time,
    MAX(datahora_ts) AS end_time,
    SUM(duration) AS total_duration,
    ST_X(geom::geometry) AS longitude,
    ST_Y(geom::geometry) AS latitude,
    grp
FROM stationary_groups
WHERE EXTRACT(HOUR FROM datahora_ts) >= 22 OR EXTRACT(HOUR FROM datahora_ts) < 5
GROUP BY ordem, linha, grp, geom
HAVING SUM(duration) >= 10800; -- 10 minutes in seconds
"""

In [141]:
df_garagem = pd.read_sql(query_garagem, conn)

/tmp/ipykernel_7675/3362709450.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_garagem = pd.read_sql(query_garagem, conn)


In [142]:
df_garagem = df_garagem.drop_duplicates(subset=['grp'])

In [143]:
pontos_de_parada = {}

In [144]:
for linha in linhas_de_interesse:
    pontos_de_parada[linha]=[]
    for _, row in df_garagem.loc[df_garagem['linha'] == linha].iterrows():
        lat, lon = row['latitude'], row['longitude']
        ponto = (lat, lon)
        encontrado = False
        for pontos in pontos_de_parada[linha]:
            if margem_erro(pontos[0], ponto, radius=300):
                encontrado = True
                pontos[1] += 1
                break
        if not encontrado:
            pontos_de_parada[linha].append([ponto, 1])
for linha in pontos_de_parada.keys():
        pontos_de_parada[linha] = sorted(pontos_de_parada[linha], key=lambda x: x[1], reverse=True)

In [145]:
for linha in pontos_de_parada.keys():
    print(f"{linha} {pontos_de_parada[linha]}")

483 [[(-22.80418, -43.31023), 104]]
864 [[(-22.89368, -43.53259), 109], [(-22.84149, -43.25392), 2]]
639 [[(-22.81763, -43.30176), 257]]
3 [[(-22.88342, -43.49536), 221], [(-22.93548, -43.6565), 1]]
309 [[(-22.87509, -43.24074), 333]]
774 [[(-22.81756, -43.3016), 104], [(-22.84353, -43.24857), 1]]
629 [[(-22.81775, -43.30167), 213], [(-22.84388, -43.24839), 1], [(-22.75517, -43.2938), 1]]
371 [[(-22.87744, -43.36824), 24]]
397 []
100 [[(-22.88907, -43.29319), 231], [(-22.8745, -43.24212), 68], [(-22.84134, -43.25389), 4]]
838 [[(-22.89399, -43.5327), 47]]
315 [[(-22.87549, -43.24131), 110], [(-22.95217, -43.34944), 23], [(-22.81696, -43.30177), 8], [(-22.86062, -43.35616), 2]]
624 [[(-22.87622, -43.36805), 21]]
388 [[(-22.90154, -43.18066), 1], [(-22.93159, -43.6558), 1]]
918 [[(-22.89523, -43.53366), 52], [(-22.84269, -43.25409), 1]]
665 [[(-22.85928, -43.37079), 74], [(-22.8164, -43.39433), 45], [(-22.79242, -43.29449), 41], [(-22.81425, -43.38256), 8], [(-22.80611, -43.3633), 1], [(

759, 388 não apresentaram resultados conclusivos

In [146]:
pontos_garagem = {}

In [147]:
for linha in linhas_de_interesse:
    pontos_de_garagem_linha = pontos_de_parada[linha]
    if len(pontos_de_parada[linha]) > 0:
        pontos_garagem[linha] = pontos_de_garagem_linha[0][0]

In [149]:
for linha in pontos_garagem.keys():
    print(f"{linha} {pontos_garagem[linha]}")

483 (-22.80418, -43.31023)
864 (-22.89368, -43.53259)
639 (-22.81763, -43.30176)
3 (-22.88342, -43.49536)
309 (-22.87509, -43.24074)
774 (-22.81756, -43.3016)
629 (-22.81775, -43.30167)
371 (-22.87744, -43.36824)
100 (-22.88907, -43.29319)
838 (-22.89399, -43.5327)
315 (-22.87549, -43.24131)
624 (-22.87622, -43.36805)
388 (-22.90154, -43.18066)
918 (-22.89523, -43.53366)
665 (-22.85928, -43.37079)
328 (-22.81515, -43.18753)
497 (-22.80449, -43.30967)
878 (-22.88064, -43.35695)
355 (-22.81683, -43.3015)
606 (-22.90259, -43.29907)
457 (-22.88888, -43.29289)
550 (-22.94975, -43.3482)
803 (-22.87612, -43.41912)
917 (-22.87654, -43.36839)
638 (-22.86042, -43.356)
2336 (-22.91719, -43.6077)
399 (-22.81743, -43.39355)
298 (-22.8162, -43.39447)
867 (-22.89435, -43.53227)
553 (-22.95321, -43.34991)
565 (-22.95077, -43.34814)
422 (-22.88955, -43.29165)
756 (-22.87659, -43.41991)
292 (-22.86867, -43.29274)
554 (-22.95263, -43.34921)
634 (-22.81524, -43.18743)
232 (-22.90244, -43.29907)
415 (-22.9

In [151]:
query_garagem = """
WITH ordered_points AS (
    SELECT
        ordem,
        linha,
        geom,
        TO_TIMESTAMP(datahora / 1000) AS datahora_ts,
        LAG(TO_TIMESTAMP(datahora / 1000)) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_datahora_ts,
        LAG(geom) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_geom,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS rn
    FROM combined_table
),
same_position_periods AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        EXTRACT(EPOCH FROM (datahora_ts - prev_datahora_ts)) AS duration,
        CASE 
            WHEN ST_DWithin(geom, prev_geom, 15) THEN 1
            ELSE 0
        END AS is_stationary,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY datahora_ts) - 
        ROW_NUMBER() OVER (PARTITION BY ordem, CASE WHEN ST_DWithin(geom, prev_geom, 15) THEN 1 ELSE 0 END ORDER BY datahora_ts) AS grp
    FROM ordered_points
),
stationary_groups AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        duration,
        is_stationary,
        grp
    FROM same_position_periods
    WHERE is_stationary = 1
)
SELECT
    ordem,
    linha,
    MIN(prev_datahora_ts) AS start_time,
    MAX(datahora_ts) AS end_time,
    SUM(duration) AS total_duration,
    ST_X(geom::geometry) AS longitude,
    ST_Y(geom::geometry) AS latitude,
    grp
FROM stationary_groups
GROUP BY ordem, linha, grp, geom
HAVING SUM(duration) >= 10800; -- 10 minutes in seconds
"""

In [152]:
df_garagem = pd.read_sql(query_garagem, conn)
df_garagem = df_garagem.drop_duplicates(subset=['grp'])

/tmp/ipykernel_7675/1241853423.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_garagem = pd.read_sql(query_garagem, conn)


In [153]:
pontos_de_parada = {}

In [154]:
for linha in linhas_de_interesse:
    pontos_de_parada[linha]=[]
    for _, row in df_garagem.loc[df_garagem['linha'] == linha].iterrows():
        lat, lon = row['latitude'], row['longitude']
        ponto = (lat, lon)
        encontrado = False
        for pontos in pontos_de_parada[linha]:
            if margem_erro(pontos[0], ponto, radius=300):
                encontrado = True
                pontos[1] += 1
                break
        if not encontrado:
            pontos_de_parada[linha].append([ponto, 1])
for linha in pontos_de_parada.keys():
        pontos_de_parada[linha] = sorted(pontos_de_parada[linha], key=lambda x: x[1], reverse=True)

In [155]:
for linha in pontos_de_parada.keys():
    print(f"{linha} {pontos_de_parada[linha]}")

483 [[(-22.80451, -43.30962), 106], [(-22.98569, -43.19711), 21], [(-22.8366, -43.28367), 2], [(-22.87811, -43.24129), 1]]
864 [[(-22.89444, -43.5319), 20], [(-22.90166, -43.55583), 4], [(-22.87556, -43.46373), 1]]
639 [[(-22.81695, -43.30219), 138], [(-22.92337, -43.23484), 1]]
3 [[(-22.88376, -43.49459), 216], [(-22.90203, -43.55459), 1]]
309 [[(-22.87528, -43.24152), 147], [(-22.9056, -43.19701), 7], [(-22.90469, -43.19267), 2], [(-23.00173, -43.3653), 1], [(-23.01087, -43.36702), 1]]
774 [[(-22.81675, -43.30168), 12], [(-22.84367, -43.2487), 2], [(-22.80716, -43.32727), 1]]
629 [[(-22.81684, -43.30185), 45], [(-22.84377, -43.2544), 2], [(-22.75474, -43.29387), 2], [(-22.84383, -43.24844), 1]]
371 [[(-22.87688, -43.36804), 1]]
397 [[(-22.9021, -43.55465), 27], [(-22.88383, -43.49508), 1]]
100 [[(-22.88891, -43.29319), 118], [(-22.87486, -43.24226), 47], [(-22.84134, -43.25389), 8], [(-22.90499, -43.192), 1], [(-22.90433, -43.1779), 1]]
838 [[(-22.89528, -43.53214), 19], [(-22.90158,

In [157]:
pontos_garagem['759'] = (-22.93569, -43.65591)
pontos_garagem['388'] = (-22.93543, -43.65633)

In [3]:
arquivo = 'dicionarios.txt'

In [158]:
with open(arquivo, 'w') as f:
    json.dump({'dicionario1': pontos_finais, 'dicionario2': pontos_garagem}, f, indent=4)

In [16]:
with open(arquivo, 'r') as f:
    dados = json.load(f)

In [17]:
pontos_finais = dados['dicionario1']
pontos_garagem = dados['dicionario2']

In [29]:
fatia = pd.read_sql("SELECT * FROM combined_table WHERE linha = '292'", conn)

/tmp/ipykernel_10465/601315889.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fatia = pd.read_sql("SELECT * FROM combined_table WHERE linha = '292'", conn)


In [30]:
fatia.columns

Index(['id', 'ordem', 'latitude', 'longitude', 'datahora', 'velocidade',
       'linha', 'datahoraenvio', 'datahoraservidor', 'geom', 'unique_id',
       'sentido'],
      dtype='object')

In [37]:
fatia['sentido'] = -1
ordens = fatia['ordem'].unique()
ultimo_final =  0
ultimo_garagem = 0
to_delete = []
for ordem in ordens:
    filtrados = fatia[fatia['ordem'] == ordem].sort_values(by='datahora')
    for index, row in filtrados.iterrows():
        ponto = (row['latitude'], row['longitude'])
        if margem_erro(ponto, pontos_finais[row['linha']][0]):
            fatia.loc[filtrados.index[ultimo_final:index+1], 'sentido'] = 0
            ultimo_final = index
            print(f"Atualizando sentido para 0 de {ultimo_final} a {index + 1} para ordem {ordem}")
        elif margem_erro(ponto, pontos_finais[row['linha']][1], radius = 200):
            fatia.loc[filtrados.index[ultimo_final:index+1], 'sentido'] = 1
            ultimo_final = index
            print(f"Atualizando sentido para 1 de {ultimo_final} a {index + 1} para ordem {ordem}")
        elif margem_erro(ponto, pontos_garagem[row['linha']]):
            to_delete += fatia.loc[filtrados.index[ultimo_final:index+1], 'unique_id'].tolist()
            ultimo_final = index
            print(f"Adicionando para excluir de {ultimo_final} a {index + 1} para ordem {ordem}")
fatia = fatia[~fatia['unique_id'].isin(to_delete)]

Atualizando sentido para 1 de 338920 a 338921 para ordem A29070
Atualizando sentido para 1 de 338984 a 338985 para ordem A29070
Atualizando sentido para 1 de 339051 a 339052 para ordem A29070
Atualizando sentido para 1 de 339134 a 339135 para ordem A29070
Atualizando sentido para 1 de 339224 a 339225 para ordem A29070
Atualizando sentido para 1 de 339291 a 339292 para ordem A29070
Atualizando sentido para 1 de 343274 a 343275 para ordem A29070
Atualizando sentido para 1 de 343343 a 343344 para ordem A29070
Atualizando sentido para 1 de 343398 a 343399 para ordem A29070
Atualizando sentido para 1 de 343462 a 343463 para ordem A29070
Atualizando sentido para 1 de 339534 a 339535 para ordem A29070
Atualizando sentido para 1 de 339613 a 339614 para ordem A29070
Atualizando sentido para 1 de 339690 a 339691 para ordem A29070
Atualizando sentido para 1 de 339769 a 339770 para ordem A29070
Atualizando sentido para 1 de 339855 a 339856 para ordem A29070
Atualizando sentido para 1 de 339941 a 3

In [40]:
fatia['sentido'].value_counts()

sentido
-1    289537
 1     90487
 0     47218
Name: count, dtype: int64

In [39]:
conn.commit()
#conn.close()